In [1]:
import tszip
import pandas as pd
import re
import sc2ts

## Get matching results for Pango X not in the ARG

https://github.com/jeromekelleher/sc2ts-paper/issues/366

In [2]:
metadata_file = "../data/run_metadata.v04.tsv"
ds_file = "../data/viridian_mafft_2024-10-14_v1.vcz.zip"
ts_file = "../data/v1-beta1_2023-02-21.pp.md.ts.dated.il.tsz"
matches_file = "../data/v1-beta1.matches.tsv"

In [3]:
metadata = pd.read_csv(metadata_file, sep="\t")
metadata = metadata[
    (metadata["Viridian_result"] == "PASS") & \
    (metadata["In_Viridian_tree"] == "T") & \
    (metadata["Date_tree"] != ".") & \
    (metadata["Date_tree"] != "2020/12/31")
]
metadata.head(1)

/var/folders/4h/9ylb643n37d8jc7srjm7qzk00000gr/T/ipykernel_59908/224125354.py:1: DtypeWarning: Columns (16,18,19,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(metadata_file, sep="\t")


,Study,Sample,Experiment,Run,Platform,Country,Region,Collection_date,First_created,Date_tree,...,Viridian_amplicon_scheme,Viridian_N,Genbank_N,Viridian_pangolin,Viridian_scorpio,Genbank_pangolin,Genbank_scorpio,Genbank_tree_name,Viridian_cons_len,Viridian_cons_het
0,PRJEB47121,SAMEA9781395,ERX6172603,ERR6546375,ILLUMINA,Estonia,none,2021-08-02,2021-08-22,2021-08-02,...,COVID-ARTIC-V3,306,.,AY.100,Delta (B.1.617.2-like),.,.,.,29810,0


In [4]:
ts = tszip.decompress(ts_file)
ts

In [5]:
matches = pd.read_csv(matches_file, sep="\t")
matches.head(1)

,strain,n_parents,n_mutations,parents,mutations
0,SRR14631544,1,24,"(0:29904, 1)","[C241T, C3037T, C3267T, C4683T, C5986T, C6471T..."


In [6]:
def parse_parents(s):
    pattern = r"\((\d+):(\d+), (\d+)\)"
    matches = re.findall(pattern, s)
    result = [
        (int(segment_start), int(segment_end), int(parent_id))
        for segment_start, segment_end, parent_id in matches
    ]
    return result


def get_pango_parents(node_id, ts):
    node = ts.node(id_=node_id)
    return node.metadata.get(
        "Viridian_pangolin",
        node.metadata.get(
            "Imputed_Viridian_pangolin",
            None,
        )
    )


def get_info_for(pango, ts):
    samples = metadata[metadata["Viridian_pangolin"] == pango]["Run"].to_list()
    results = matches[matches["strain"].isin(samples)].reset_index(drop=True)
    print(f"Samples in Viridian: {len(samples)}")
    print(f"HMM results in matches: {len(results)}")
    data = []
    for _, row in results.iterrows():
        if row["n_parents"] == 1:
            data.append(
                {
                    "sample": row["strain"],
                    "pango": pango,
                    "recombinant_type": "non",
                    "left_pango_parent": "n/a",
                    "right_pango_parent": "n/a",
                    "left_breakpoint": "n/a",
                    "right_breakpoint": "n/a",
                    "num_mutations": row["n_mutations"],
                }
            )
        elif row["n_parents"] == 2:
            parsed_results = parse_parents(row["parents"])
            left_parent_id = parsed_results[0][2]
            right_parent_id = parsed_results[1][2]
            left_pango_parent = get_pango_parents(left_parent_id, ts)
            right_pango_parent = get_pango_parents(right_parent_id, ts)
            right_breakpoint = parsed_results[1][0]
            left_breakpoint = sc2ts.utils.compute_left_bound(
                ts=ts,
                parents=[left_parent_id, right_parent_id],
                right=right_breakpoint,
            )
            data.append(
                {
                    "sample": row["strain"],
                    "pango": pango,
                    "recombinant_type": "simple",
                    "left_pango_parent": left_pango_parent,
                    "right_pango_parent": right_pango_parent,
                    "left_breakpoint": left_breakpoint,
                    "right_breakpoint": right_breakpoint,
                    "num_mutations": row["n_mutations"],
                }
            )
        else:
            data.append(
                {
                    "sample": row["strain"],
                    "pango": pango,
                    "recombinant_type": "complex",
                    "left_pango_parent": "n/a",
                    "right_pango_parent": "n/a",
                    "left_breakpoint": "n/a",
                    "right_breakpoint": "n/a",
                    "num_mutations": row["n_mutations"],
                }
            )
    return pd.DataFrame(data)

### Get matching results from matches file

#### XBJ

In [7]:
# https://github.com/jeromekelleher/sc2ts-paper/issues/362
get_info_for(pango="XBJ.2", ts=ts)

Samples in Viridian: 1
HMM results in matches: 1


,sample,pango,recombinant_type,left_pango_parent,right_pango_parent,left_breakpoint,right_breakpoint,num_mutations
0,ERR10839097,XBJ.2,simple,BA.2.3.20,BA.5.2,23019,25810,6


In [8]:
get_info_for(pango="XBJ.4", ts=ts)

Samples in Viridian: 1
HMM results in matches: 0


""


#### XBP

In [9]:
# Pick more specific Pango label BQ.1.1.3 over BQ.1.5.
get_info_for(pango="XBP", ts=ts)

Samples in Viridian: 8
HMM results in matches: 2


,sample,pango,recombinant_type,left_pango_parent,right_pango_parent,left_breakpoint,right_breakpoint,num_mutations
0,ERR10770729,XBP,simple,BL.2,BQ.1.5,22191,22331,5
1,SRR23608970,XBP,simple,BL.2,BQ.1.1.3,22191,22331,7


#### XBS

In [10]:
# Pick more specific Pango label BQ.1.1 over BQ.1.
get_info_for(pango="XBS", ts=ts)

Samples in Viridian: 19
HMM results in matches: 15


,sample,pango,recombinant_type,left_pango_parent,right_pango_parent,left_breakpoint,right_breakpoint,num_mutations
0,ERR10749106,XBS,simple,BA.2.75,BQ.1.1,22034,22190,4
1,ERR10779088,XBS,simple,BA.2.75,BQ.1.1,22034,22190,6
2,ERR10771500,XBS,simple,BA.2.75,BQ.1.1,22034,22190,6
3,ERR10791804,XBS,simple,BA.2.75,BQ.1.1,22034,22190,5
4,ERR10791806,XBS,simple,BA.2.75,BQ.1.1,22034,22190,5
5,ERR10792087,XBS,simple,BA.2.75,BQ.1.1,22034,22190,5
6,ERR10797807,XBS,simple,BA.2.75,BQ.1.1,22034,22190,5
7,ERR10797810,XBS,simple,BA.2.75,BQ.1.1,22034,22190,5
8,ERR10797829,XBS,simple,BA.2.75,BQ.1.1,22034,22190,5
9,ERR10797888,XBS,simple,BA.2.75,BQ.1.1,22034,22190,5


#### XBW

In [11]:
get_info_for(pango="XBW", ts=ts)

Samples in Viridian: 1
HMM results in matches: 1


,sample,pango,recombinant_type,left_pango_parent,right_pango_parent,left_breakpoint,right_breakpoint,num_mutations
0,SRR23359879,XBW,simple,XBB.1.5,BQ.1.14,25417,26275,5


#### XCA

In [12]:
# Identical results across samples
get_info_for(pango="XCA", ts=ts)

Samples in Viridian: 11
HMM results in matches: 4


,sample,pango,recombinant_type,left_pango_parent,right_pango_parent,left_breakpoint,right_breakpoint,num_mutations
0,ERR10911716,XCA,simple,BA.2.75.2,BQ.1.1.1,22748,22893,10
1,SRR23607969,XCA,simple,BA.2.75.2,BQ.1.1.1,22748,22893,12
2,ERR10911045,XCA,simple,BA.2.75.2,BQ.1.1.1,22748,22893,11
3,ERR10911252,XCA,simple,BA.2.75.2,BQ.1.1.1,22748,22893,12


### Rematch samples for XAB, XAH, XBN, XBZ, and XCG

In [13]:
# Access to the trees including samples up to the day before the following dates is needed.
# Note that all the collection dates are not Dec. 31, 2020 (excluded).
from collections import defaultdict
pangos = ["XAB", "XAH", "XBN", "XBZ", "XCG"]
pango_sample_map = defaultdict(list)
for p in pangos:
    for s in metadata[metadata["Viridian_pangolin"] == p]["Run"].to_list():
        # NOTE: Not "Collection_date".
        d = metadata[metadata["Run"] == s]["Date_tree"].to_list()[0]
        pango_sample_map[p].append({"sample": s, "date": d})
pango_sample_map

defaultdict(list,
            {'XAB': [{'sample': 'ERR9633679', 'date': '2022-03-28'},
              {'sample': 'ERR9633687', 'date': '2022-03-28'}],
             'XAH': [{'sample': 'ERR9369123', 'date': '2022-03-13'},
              {'sample': 'SRR21197095', 'date': '2022-06-03'}],
             'XBN': [{'sample': 'SRR22249272', 'date': '2022-10-24'},
              {'sample': 'SRR22391626', 'date': '2022-10-26'}],
             'XBZ': [{'sample': 'ERR10513123', 'date': '2022-11-18'},
              {'sample': 'ERR10896878', 'date': '2023-01-30'}],
             'XCG': [{'sample': 'SRR22928207', 'date': '2022-12-06'},
              {'sample': 'SRR22483648', 'date': '2022-11-21'}]})

In [14]:
from datetime import datetime, timedelta

def get_matching_results_for(
    pango,
    pango_sample_map,
    ts_dir,
    *,
    # Use default values here, except num_mismatches.
    # https://github.com/jeromekelleher/sc2ts/blob/main/sc2ts/cli.py
    num_mismatches=4,
    mismatch_threshold=100,
    direction="forward",
):
    runs = []
    for s in pango_sample_map[pango]:
        prev_date = datetime.strptime(s['date'], '%Y-%m-%d').date() - timedelta(1)
        daily_ts_file = ''.join([ts_dir, "v1-beta1", "_", str(prev_date), ".ts"])
        runs.append(
            sc2ts.run_hmm(
                ds_file,
                daily_ts_file,
                strains=[s['sample']],
                num_mismatches=num_mismatches,
                mismatch_threshold=mismatch_threshold,
                direction=direction,
                num_threads=0,
                show_progress=True,
            )[0]
        )
    data = []
    for run in runs:
        result = run.asdict()
        sample = result["strain"]
        num_segments = len(result["match"]["path"])
        num_mutations = len(result["match"]["mutations"])
        hmm_cost = result["match"]["cost"]
        if num_segments == 1:
            data.append(
                {
                    "sample": sample,
                    "pango": pango,
                    "recombinant_type": "non",
                    "left_pango_parent": "n/a",
                    "right_pango_parent": "n/a",
                    "left_breakpoint": "n/a",
                    "right_breakpoint": "n/a",
                    "num_mutations": num_mutations,
                    "hmm_cost": hmm_cost,
                }
            )
        elif num_segments == 2:
            left_parent_id = result["match"]["path"][0]["parent"]
            right_parent_id = result["match"]["path"][1]["parent"]
            left_pango_parent = get_pango_parents(left_parent_id, ts)
            right_pango_parent = get_pango_parents(right_parent_id, ts)
            right_breakpoint = result["match"]["path"][1]["left"]
            left_breakpoint = sc2ts.utils.compute_left_bound(
                ts=ts,
                parents=[left_parent_id, right_parent_id],
                right=right_breakpoint,
            )
            data.append(
                {
                    "sample": sample,
                    "pango": pango,
                    "recombinant_type": "simple",
                    "left_pango_parent": left_pango_parent,
                    "right_pango_parent": right_pango_parent,
                    "left_breakpoint": left_breakpoint,
                    "right_breakpoint": right_breakpoint,
                    "num_mutations": num_mutations,
                    "hmm_cost": hmm_cost,
                }
            )
        else:
            data.append(
                {
                    "sample": sample,
                    "pango": pango,
                    "recombinant_type": "complex",
                    "left_pango_parent": "n/a",
                    "right_pango_parent": "n/a",
                    "left_breakpoint": "n/a",
                    "right_breakpoint": "n/a",
                    "num_mutations": num_mutations,
                    "hmm_cost": hmm_cost,
                }
            )
    return pd.DataFrame(data)

#### XAB

In [15]:
# Both samples have >500 missing sites, so filtered out by sc2ts QC.
# Non-recombinant
get_matching_results_for(pango="XAB", pango_sample_map=pango_sample_map, ts_dir="../inference/")

Match:preprocess      100%|██████████| 1.00/1.00 [00:00, 2.80it/s]
Match:HMM             100%|██████████| 1.00/1.00 [00:58, 59.0s/it]
Match:preprocess      100%|██████████| 1.00/1.00 [00:00, 3.66it/s]
Match:HMM             100%|██████████| 1.00/1.00 [00:58, 58.5s/it]


,sample,pango,recombinant_type,left_pango_parent,right_pango_parent,left_breakpoint,right_breakpoint,num_mutations,hmm_cost
0,ERR9633679,XAB,non,n/a,n/a,n/a,n/a,57,57
1,ERR9633687,XAB,non,n/a,n/a,n/a,n/a,57,57


#### XAH

In [16]:
# Both samples have >500 missing sites, so filtered out by sc2ts QC.
# Non-recombinant
get_matching_results_for(pango="XAH", pango_sample_map=pango_sample_map, ts_dir="../inference/")

Match:preprocess      100%|██████████| 1.00/1.00 [00:00, 3.60it/s]
Match:HMM             100%|██████████| 1.00/1.00 [00:54, 54.9s/it]
Match:preprocess      100%|██████████| 1.00/1.00 [00:00, 3.80it/s]
Match:HMM             100%|██████████| 1.00/1.00 [01:37, 97.3s/it]


,sample,pango,recombinant_type,left_pango_parent,right_pango_parent,left_breakpoint,right_breakpoint,num_mutations,hmm_cost
0,ERR9369123,XAH,non,n/a,n/a,n/a,n/a,31,31
1,SRR21197095,XAH,non,n/a,n/a,n/a,n/a,36,36


#### XBN

In [17]:
# See https://github.com/cov-lineages/pango-designation/blob/master/pango_designation/alias_key.json
# BA = B.1.1.529
# BN = B.1.1.529.2.75.5
#
# Pick more specific left Pango parent BN.1.
get_matching_results_for(pango="XBN", pango_sample_map=pango_sample_map, ts_dir="../inference/")

Match:preprocess      100%|██████████| 1.00/1.00 [00:00, 3.60it/s]
Match:HMM             100%|██████████| 1.00/1.00 [00:42, 42.1s/it]
Match:preprocess      100%|██████████| 1.00/1.00 [00:00, 4.38it/s]
Match:HMM             100%|██████████| 1.00/1.00 [01:09, 69.4s/it]


,sample,pango,recombinant_type,left_pango_parent,right_pango_parent,left_breakpoint,right_breakpoint,num_mutations,hmm_cost
0,SRR22249272,XBN,simple,BA.2.75.5,XBB,5184,12444,61,65
1,SRR22391626,XBN,simple,BN.1,XBB,5184,12444,60,64


#### XBZ

In [18]:
# Non-recombinant
get_matching_results_for(pango="XBZ", pango_sample_map=pango_sample_map, ts_dir="../inference/")

Match:preprocess      100%|██████████| 1.00/1.00 [00:00, 3.51it/s]
Match:HMM             100%|██████████| 1.00/1.00 [01:15, 75.9s/it]
Match:preprocess      100%|██████████| 1.00/1.00 [00:00, 3.91it/s]
Match:HMM             100%|██████████| 1.00/1.00 [01:28, 88.7s/it]


,sample,pango,recombinant_type,left_pango_parent,right_pango_parent,left_breakpoint,right_breakpoint,num_mutations,hmm_cost
0,ERR10513123,XBZ,non,n/a,n/a,n/a,n/a,67,67
1,ERR10896878,XBZ,non,n/a,n/a,n/a,n/a,68,68


#### XCG

In [19]:
# Non-recombinant
get_matching_results_for(pango="XCG", pango_sample_map=pango_sample_map, ts_dir="../inference/")

Match:preprocess      100%|██████████| 1.00/1.00 [00:00, 4.27it/s]
Match:HMM             100%|██████████| 1.00/1.00 [01:41, 102s/it]
Match:preprocess      100%|██████████| 1.00/1.00 [00:00, 4.48it/s]
Match:HMM             100%|██████████| 1.00/1.00 [01:41, 102s/it]


,sample,pango,recombinant_type,left_pango_parent,right_pango_parent,left_breakpoint,right_breakpoint,num_mutations,hmm_cost
0,SRR22928207,XCG,non,n/a,n/a,n/a,n/a,69,69
1,SRR22483648,XCG,non,n/a,n/a,n/a,n/a,61,61
